Relevant searches to extract data structures that can be of use down the line.

In [1]:
import pandas as pd
import json
import sqlite3
import numpy as np
from datetime import datetime

In [2]:
conn = sqlite3.connect('../data/recsys.db')
c = conn.cursor()

# Nutrient Data

1. Load the nutrient data and pivot (long => wide).
2. Then join this with the original recipe data. This can be saved out and sent.

In [21]:
df = pd.read_sql_query('''SELECT * FROM nutrients;''', conn)
df.head(30)

,nutrient_id,recipe_id,complete,name,value,percent_daily_value,display_value,value_unit
0,1,222388,0,Niacin Eqivalents,9.319291,72.0,9,mg
1,2,222388,1,Sgars,0.093559,0.0,0.1,g
2,3,222388,1,Sodim,2017.130000,81.0,2017,mg
3,4,222388,1,Carbohydrates,1.797819,0.0,1.8,g
4,5,222388,0,Vitamin B6,0.232980,15.0,< 1,mg
5,6,222388,1,Calories,308.148100,15.0,308,kcal
6,7,222388,0,Thiamin,0.394774,39.0,< 1,mg
7,8,222388,1,Fat,23.585870,36.0,23.6,g
8,9,222388,0,Folate,2.109131,1.0,2,mcg
9,10,222388,1,Calories from Fat,212.272800,NaN,212,kcal


In [6]:
df_val = df.pivot(index='recipe_id', columns='name', values='value')
df_val.head()

name,Calcim,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesim,Niacin Eqivalents,Potassim,Protein,Satrated Fat,Sgars,Sodim,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C
recipe_id,,,,,,,,,,,,,,,,,,,,
6663,20.955480,120.7066,74.32346,9.370335,1.466667,0.329367,8.258162,16.15505,0.559310,4.213947,1.305051,18.26903,2.300013,2.257850,0.190441,78.67492,0.082433,7.767053,0.007098,0.024194
6664,210.903400,313.8263,138.84220,35.498860,85.738990,2.394961,15.426910,46.72293,1.778180,36.901050,2.932469,196.93310,9.789059,8.499446,7.169690,623.72670,0.212935,876.401900,0.144638,4.720046
6665,6.107055,104.0672,26.76795,16.622390,10.333330,0.597292,2.974216,45.38445,1.002785,5.176659,1.801886,33.46304,2.499005,0.551574,1.951010,163.35680,0.161123,124.124600,0.018682,0.007361
6666,18.133790,303.3300,121.62470,43.189830,31.000000,1.555075,13.513860,39.37596,1.242923,15.293330,2.180518,129.81990,3.694001,1.792503,26.388880,171.38830,0.190914,3246.537000,0.077493,3.324017
6667,46.995660,197.9225,32.90274,34.942120,9.495556,1.218333,3.655860,76.35577,2.018598,13.394780,2.921615,105.10400,5.619544,2.114157,2.975532,97.72266,0.350947,142.911800,0.038195,0.069111


In [13]:
# Only get those recipes with some sort-of nutrient info
df_recipes = pd.read_sql_query('''SELECT * FROM recipes WHERE id in (SELECT DISTINCT(recipe_id) FROM nutrients);''', conn)
df_recipes.head()

,id,name,ave_rating,image_url,review_nums,ingredients,directions
0,222388,Homemade Bacon,5.000000,https://images.media-allrecipes.com/userphotos...,3,"pork belly, smoked paprika, kosher salt, groun...",'Prep\n5 m\nCook\n2 h 45 m\nReady In\n11 h 50 ...
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,https://images.media-allrecipes.com/userphotos...,29,"sauerkraut drained, Granny Smith apples sliced...",'Prep\n15 m\nCook\n2 h 30 m\nReady In\n2 h 45 ...
2,218939,Foolproof Rosemary Chicken Wings,4.571429,https://images.media-allrecipes.com/userphotos...,12,"chicken wings, sprigs rosemary, head garlic, o...","""Prep\n20 m\nCook\n40 m\nReady In\n1 h\nPrehea..."
3,87211,Chicken Pesto Paninis,4.625000,https://images.media-allrecipes.com/userphotos...,163,"focaccia bread quartered, prepared basil pesto...",'Prep\n15 m\nCook\n5 m\nReady In\n20 m\nPrehea...
4,245714,Potato Bacon Pizza,4.500000,https://images.media-allrecipes.com/userphotos...,2,"red potatoes, strips bacon, Sauce:, heavy whip...",'Prep\n20 m\nCook\n45 m\nReady In\n1 h 10 m\nP...


In [7]:
pd.merge(df, df_recipes, )

name,Calcim,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesim,Niacin Eqivalents,Potassim,Protein,Satrated Fat,Sgars,Sodim,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C
recipe_id,,,,,,,,,,,,,,,,,,,,
262646,84.40975,113.2842,18.68494,24.814770,0.0000,0.274675,2.076105,41.05075,1.164613,41.04175,3.748584,369.0890,2.434993,0.403351,0.750000,1044.9860,0.126048,425.3670,0.344493,7.188650
262797,68.69238,307.3315,118.29820,22.008190,64.6000,2.953104,13.144250,58.90721,2.351311,54.36844,16.624860,547.9546,26.052780,2.203500,14.878050,782.9474,0.189611,1690.5150,0.668266,56.748460
262815,25.25903,235.1897,133.69730,4.055043,52.0725,0.271827,14.855250,14.45597,1.576513,20.14212,7.895879,302.2780,20.457030,4.529202,0.516253,115.6808,0.119869,108.4522,0.423013,0.953285
262850,70.08701,551.5228,313.05810,6.597518,114.1225,1.474026,34.784240,15.42799,3.593588,39.75600,11.674960,516.5079,31.451040,12.495910,0.829528,165.0731,0.140095,677.9331,0.687440,4.074295
262956,161.18040,734.9250,408.33760,54.675650,132.0595,4.300613,45.370840,204.86310,4.990212,64.67340,8.357469,550.9448,18.200980,22.930110,5.167469,613.4282,0.773093,2344.3600,0.324647,9.982000


In [18]:
df2 = pd.merge(df_recipes, df_val, left_on='id', right_on='recipe_id')
df2.head()

,id,name,ave_rating,image_url,review_nums,ingredients,directions,Calcim,Calories,Calories from Fat,...,Niacin Eqivalents,Potassim,Protein,Satrated Fat,Sgars,Sodim,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C
0,222388,Homemade Bacon,5.000000,https://images.media-allrecipes.com/userphotos...,3,"pork belly, smoked paprika, kosher salt, groun...",'Prep\n5 m\nCook\n2 h 45 m\nReady In\n11 h 50 ...,11.18365,308.1481,212.27280,...,9.319291,347.2267,21.002540,7.736815,0.093559,2017.1300,0.394774,474.20730,0.232980,0.776127
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,https://images.media-allrecipes.com/userphotos...,29,"sauerkraut drained, Granny Smith apples sliced...",'Prep\n15 m\nCook\n2 h 30 m\nReady In\n2 h 45 ...,135.45380,371.7219,105.07690,...,15.601600,1088.9230,36.398780,3.646474,19.841460,2606.7640,0.842312,73.17785,1.328631,52.768480
2,218939,Foolproof Rosemary Chicken Wings,4.571429,https://images.media-allrecipes.com/userphotos...,12,"chicken wings, sprigs rosemary, head garlic, o...","""Prep\n20 m\nCook\n40 m\nReady In\n1 h\nPrehea...",60.08832,335.1655,212.62670,...,10.066790,249.1827,23.912650,5.683611,0.236130,762.8050,0.064960,359.36400,0.553800,5.307448
3,87211,Chicken Pesto Paninis,4.625000,https://images.media-allrecipes.com/userphotos...,163,"focaccia bread quartered, prepared basil pesto...",'Prep\n15 m\nCook\n5 m\nReady In\n20 m\nPrehea...,528.46170,640.5617,264.30850,...,13.228260,348.8067,32.375370,10.918760,1.974210,1075.5270,0.030880,604.75370,0.273496,18.015020
4,245714,Potato Bacon Pizza,4.500000,https://images.media-allrecipes.com/userphotos...,2,"red potatoes, strips bacon, Sauce:, heavy whip...",'Prep\n20 m\nCook\n45 m\nReady In\n1 h 10 m\nP...,132.22650,162.6685,68.40259,...,3.013383,106.8477,7.059566,3.975159,2.452803,189.8088,0.165943,168.32450,0.055718,0.905797


In [19]:
df2.shape

(48632, 27)

In [20]:
df2.to_csv('../data/14_recipes_with_nutrients.csv')